# Tích hợp Azure AI Search với NVIDIA NIM và LlamaIndex

Trong notebook này, chúng ta sẽ trình bày cách tận dụng các mô hình AI của NVIDIA và LlamaIndex để tạo ra một pipeline Retrieval-Augmented Generation (RAG) mạnh mẽ. Chúng ta sẽ sử dụng các LLMs và embeddings của NVIDIA, tích hợp chúng với Azure AI Search làm kho lưu trữ vector, và thực hiện RAG để nâng cao chất lượng và hiệu quả tìm kiếm.

## Lợi ích
- **Khả năng mở rộng**: Tận dụng các mô hình ngôn ngữ lớn của NVIDIA và Azure AI Search để tìm kiếm hiệu quả và có khả năng mở rộng.
- **Hiệu quả chi phí**: Tối ưu hóa tìm kiếm và truy xuất với lưu trữ vector hiệu quả và các kỹ thuật tìm kiếm kết hợp.
- **Hiệu suất cao**: Kết hợp các LLMs mạnh mẽ với tìm kiếm vector hóa để có phản hồi nhanh hơn và chính xác hơn.
- **Chất lượng**: Duy trì chất lượng tìm kiếm cao bằng cách dựa vào các tài liệu liên quan được truy xuất để hỗ trợ phản hồi của LLM.

## Yêu cầu
- 🐍 Python 3.9 hoặc cao hơn
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 API Key của NVIDIA để truy cập các LLMs và Embeddings của NVIDIA thông qua các microservices của NVIDIA NIM

## Các tính năng được đề cập
- ✅ Tích hợp NVIDIA LLM (chúng ta sẽ sử dụng [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Embeddings của NVIDIA (chúng ta sẽ sử dụng [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Các chế độ truy xuất nâng cao của Azure AI Search
- ✅ Lập chỉ mục tài liệu với LlamaIndex
- ✅ RAG sử dụng Azure AI Search và LlamaIndex với các LLMs của NVIDIA

Hãy bắt đầu nào!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Cài đặt và yêu cầu
Tạo môi trường Python sử dụng phiên bản Python >3.10.

## Bắt đầu!


Để bắt đầu, bạn cần một `NVIDIA_API_KEY` để sử dụng các mô hình NVIDIA AI Foundation:  
1) Tạo một tài khoản miễn phí với [NVIDIA](https://build.nvidia.com/explore/discover).  
2) Nhấp vào mô hình bạn chọn.  
3) Trong phần Input, chọn tab Python, sau đó nhấp vào **Get API Key** và tiếp tục nhấp **Generate Key**.  
4) Sao chép và lưu lại khóa được tạo dưới dạng NVIDIA_API_KEY. Từ đó, bạn sẽ có quyền truy cập vào các điểm cuối.  


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Ví dụ RAG sử dụng LLM và Embedding
### 1) Khởi tạo LLM
`llama-index-llms-nvidia`, còn được gọi là trình kết nối LLM của NVIDIA, cho phép bạn kết nối và tạo từ các mô hình tương thích có sẵn trong danh mục API của NVIDIA. Xem danh sách các mô hình hoàn thành hội thoại tại đây: https://build.nvidia.com/search?term=Text-to-Text

Ở đây, chúng ta sẽ sử dụng **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Khởi tạo Embedding
`llama-index-embeddings-nvidia`, còn được gọi là NVIDIA's Embeddings connector, cho phép bạn kết nối và tạo từ các mô hình tương thích có sẵn trong danh mục API của NVIDIA. Chúng tôi đã chọn `nvidia/nv-embedqa-e5-v5` làm mô hình embedding. Xem danh sách các mô hình embedding văn bản tại đây: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Tạo một Công cụ Truy vấn để đặt câu hỏi về dữ liệu của bạn

Dưới đây là một truy vấn sử dụng tìm kiếm vector thuần túy trong Azure AI Search và liên kết câu trả lời với LLM của chúng tôi (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Đây là một truy vấn sử dụng tìm kiếm lai trong Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Phân Tích Tìm Kiếm Vector
Phản hồi của LLM đã nắm bắt chính xác các hậu quả kinh tế quan trọng được đề cập trong văn bản nguồn liên quan đến thị trường chứng khoán của Nga. Cụ thể, nó nêu rằng thị trường chứng khoán Nga đã giảm mạnh, mất 40% giá trị, và giao dịch bị đình chỉ do tình hình đang diễn ra. Phản hồi này phù hợp với thông tin được cung cấp trong nguồn, cho thấy LLM đã xác định và tóm tắt đúng các chi tiết liên quan đến tác động lên thị trường chứng khoán do hành động của Nga và các lệnh trừng phạt áp đặt.

#### Bình Luận Về Các Nút Nguồn
Các nút nguồn cung cấp một bản tường thuật chi tiết về những hậu quả kinh tế mà Nga phải đối mặt do các lệnh trừng phạt quốc tế. Văn bản nhấn mạnh rằng thị trường chứng khoán Nga đã mất 40% giá trị, và giao dịch bị đình chỉ. Ngoài ra, nó còn đề cập đến các tác động kinh tế khác, như sự mất giá của đồng Ruble và sự cô lập rộng hơn của nền kinh tế Nga. Phản hồi của LLM đã chắt lọc hiệu quả các điểm quan trọng từ những nút này, tập trung vào tác động lên thị trường chứng khoán theo yêu cầu của truy vấn.


Bây giờ, hãy xem xét một truy vấn mà Tìm kiếm Lai không đưa ra câu trả lời hợp lý:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Tìm kiếm Kết hợp: Phân tích Phản hồi từ LLM
Phản hồi từ LLM trong ví dụ Tìm kiếm Kết hợp cho thấy rằng ngữ cảnh được cung cấp không nêu rõ ngày chính xác Nga xâm lược Ukraine. Phản hồi này gợi ý rằng LLM đang tận dụng thông tin có trong các tài liệu nguồn nhưng thừa nhận sự thiếu chi tiết chính xác trong văn bản.

Phản hồi này chính xác khi xác định rằng ngữ cảnh đề cập đến các sự kiện liên quan đến sự xâm lược của Nga nhưng không chỉ rõ ngày xâm lược cụ thể. Điều này thể hiện khả năng của LLM trong việc hiểu thông tin được cung cấp đồng thời nhận ra những khoảng trống trong nội dung. LLM hiệu quả trong việc gợi ý người dùng tìm kiếm các nguồn bên ngoài hoặc hồ sơ lịch sử để biết ngày chính xác, thể hiện mức độ thận trọng khi thông tin chưa đầy đủ.

### Phân tích Các Nút Nguồn
Các nút nguồn trong ví dụ Tìm kiếm Kết hợp chứa các đoạn trích từ một bài phát biểu thảo luận về phản ứng của Hoa Kỳ đối với hành động của Nga ở Ukraine. Những nút này nhấn mạnh tác động địa chính trị rộng lớn hơn và các bước mà Hoa Kỳ cùng các đồng minh đã thực hiện để đối phó với cuộc xâm lược, nhưng không đề cập đến ngày xâm lược cụ thể. Điều này phù hợp với phản hồi từ LLM, vốn đã xác định chính xác rằng ngữ cảnh thiếu thông tin về ngày chính xác.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Phân Tích Kết Hợp với Xếp Hạng Lại: Phân Tích Phản Hồi của LLM
Trong ví dụ Kết Hợp với Xếp Hạng Lại, phản hồi của LLM cung cấp thêm ngữ cảnh bằng cách lưu ý rằng sự kiện xảy ra sáu ngày trước khi bài phát biểu được đưa ra. Điều này cho thấy LLM có khả năng suy luận ngày xảy ra cuộc xâm lược dựa trên thời điểm bài phát biểu, mặc dù vẫn cần biết chính xác ngày diễn ra bài phát biểu để đảm bảo độ chính xác.

Phản hồi này thể hiện khả năng cải thiện trong việc sử dụng các manh mối ngữ cảnh để đưa ra câu trả lời mang tính thông tin hơn. Nó nhấn mạnh lợi thế của xếp hạng lại, nơi LLM có thể truy cập và ưu tiên thông tin liên quan hơn để đưa ra một ước tính gần hơn với chi tiết mong muốn (tức là ngày xảy ra cuộc xâm lược).

### Phân Tích Các Nút Nguồn
Các nút nguồn trong ví dụ này bao gồm các tham chiếu đến thời điểm Nga xâm lược, cụ thể là đề cập rằng sự kiện xảy ra sáu ngày trước bài phát biểu. Mặc dù ngày chính xác vẫn chưa được nêu rõ, các nút cung cấp ngữ cảnh thời gian cho phép LLM đưa ra phản hồi tinh tế hơn. Việc đưa vào chi tiết này cho thấy cách xếp hạng lại có thể cải thiện khả năng của LLM trong việc trích xuất và suy luận thông tin từ ngữ cảnh được cung cấp, dẫn đến một phản hồi chính xác và mang tính thông tin hơn.


**Lưu ý:**  
Trong notebook này, chúng tôi đã sử dụng các dịch vụ vi mô NVIDIA NIM từ NVIDIA API Catalog.  
Các API trên, `NVIDIA (llms)`, `NVIDIAEmbedding`, và [Azure AI Search Semantic Hybrid Retrieval (built-in reranking)](https://learn.microsoft.com/azure/search/semantic-search-overview). Lưu ý rằng các API trên cũng có thể hỗ trợ các dịch vụ vi mô tự lưu trữ.  

**Ví dụ:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm về bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
